In [1]:
import flywheel
import os
import datetime
from dateutil.tz import tzutc

fw = flywheel.Client()
project_id = '60fef55e60ec55d1b0e0741e'
local_dir = '../csvs'
os.makedirs(local_dir, exist_ok=True)

In [2]:
# Get all the sessions within the project.
sessions = [fw.get_session(x.id) for x in fw.get_project_sessions(project_id)]

In [3]:
for session in sessions:
    sub = session.subject
    if not 'Normal' in sub.tags:
        continue
        
    print(session.label)
    
    for analysis in session.analyses:
        if analysis.label.startswith('neuroprint') and analysis.job['state']=='complete' and analysis.files:
            print(analysis.label)
            csv_file = [f for f in analysis.files if f.name.endswith('csv') and 'wscore' in f.name][0]
            try:
                print("Downloading heatmaps...")
                analysis.download_file(csv_file.name, os.path.join(local_dir, csv_file.name))
                #analysis.download_file(png_files[1].name, os.path.join(local_dir, png_files[1].name))
            except IndexError as e:
                print(e)
                print("Could not download csv from {}.".format(session.label))
                
    print("")

100049x20160621x3T
neuroprint_2021-12-1_15:51_WT
neuroprint_2022-2-16_16:47_WT

100113x20160620x3T
neuroprint_2021-12-1_15:51_WT
neuroprint_2022-2-16_16:47_WT

122216x20200205x3T
neuroprint_2021-12-1_15:51_WT
neuroprint_2022-2-16_16:47_WT

125687x20200714x3T
neuroprint_2021-12-1_15:51_WT
neuroprint_2022-2-16_16:47_WT

112286x01x20200721x3T
neuroprint_2021-12-1_15:51_WT
neuroprint_2022-2-16_16:47_WT

124509x20200813x3T
neuroprint_2021-12-1_15:51_WT
neuroprint_2022-2-16_16:47_WT

107486x20200929x3T
neuroprint_2021-12-1_15:51_WT
neuroprint_2022-2-16_16:47_WT

125564x20201105x3T
neuroprint_2021-12-1_15:51_WT
neuroprint_2022-2-16_16:47_WT

123831x20201201x3T
neuroprint_2021-12-1_15:51_WT
neuroprint_2022-2-16_16:47_WT

123575x20201015x3T
neuroprint_2021-12-1_15:51_WT
neuroprint_2022-2-16_16:47_WT

126022x20210112x3T
neuroprint_2021-12-1_15:51_WT
neuroprint_2022-2-16_16:47_WT

124822x20210126x3T
neuroprint_2021-12-1_15:51_WT
neuroprint_2022-2-16_16:47_WT

125780x20210216x3T
neuroprint_2021-12

## Concatenate CSVs

In [47]:
from pathlib import Path
import pandas as pd
sheets = Path(local_dir).glob("*.csv")
sheets_strings = [str(p) for p in sheets]

df = pd.read_csv(sheets_strings[0])
label_nums = df['label_number'] # save label number column for later

new_col_list = []
for sheet in sheets_strings:
    sheet_df = pd.read_csv(sheet)
    wscores = sheet_df['w-score']
    column_name = os.path.basename(sheet).split('.')[0]
    
    col_df = pd.DataFrame({column_name: wscores})
    new_col_list.append(col_df)

In [48]:
new_col_list.insert(0, label_nums)
big_df = pd.concat(new_col_list,axis=1)
big_df

,label_number,sub-123575_ses-123575x20201015x3T_wscores,sub-125572_ses-125572x20210323x3T_wscores,sub-122946_ses-122946x20210427x3T_wscores,sub-122216_ses-122216x20200205x3T_wscores,sub-125687_ses-125687x20200714x3T_wscores,sub-113066x02_ses-113066x02x20210427x3T_wscores,sub-124938_ses-124938x20210527x3T_wscores,sub-100049_ses-100049x20160621x3T_wscores,sub-124822_ses-124822x20210126x3T_wscores,...,sub-125564_ses-125564x20201105x3T_wscores,sub-100113_ses-100113x20160620x3T_wscores,sub-126022_ses-126022x20210112x3T_wscores,sub-107486_ses-107486x20200929x3T_wscores,sub-124568_ses-124568x20210318x3T_wscores,sub-125987_ses-125987x20210610x3T_wscores,sub-124509_ses-124509x20200813x3T_wscores,sub-123831_ses-123831x20201201x3T_wscores,sub-125780_ses-125780x20210216x3T_wscores,sub-124972_ses-124972x20210525x3T_wscores
0,1,1.076187,-0.332417,1.061978,0.193376,0.897047,1.204015,0.240755,0.561843,0.273479,...,1.867585,0.842100,0.642428,0.966931,1.258821,0.686097,0.182824,1.255567,0.244671,1.506451
1,2,1.591047,-0.381592,1.211494,0.045984,0.325528,1.833228,0.340384,0.753842,0.206496,...,1.436148,0.010676,0.102841,1.003862,1.987458,0.028908,0.471241,0.597593,1.233789,1.612831
2,3,1.041013,-1.228473,-0.029155,0.567004,-0.914842,0.399388,-0.052584,0.380521,0.238076,...,0.303187,-0.345732,-0.663098,0.943625,0.644774,0.633531,-0.463397,0.396978,0.756016,0.526921
3,4,1.687107,0.768331,-0.101368,0.699904,0.786834,1.914344,1.743214,1.613962,-0.179738,...,0.796367,-0.131818,-0.139502,1.358910,1.498293,0.768210,1.439247,0.868791,0.939528,1.453562
4,5,1.352005,-0.170654,0.654741,0.808473,0.588508,2.440852,1.115450,1.475046,0.394174,...,1.960889,0.531567,0.503132,0.945649,1.423793,0.950177,0.946502,0.741275,1.394902,2.066618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,-0.232103,-0.125613,-0.202094,-0.436980,0.842365,0.214154,0.201752,0.410684,-0.325421,...,-0.025636,-0.747875,-0.465381,-0.355389,0.682459,0.292138,-0.917238,0.863410,-0.461884,0.136985
196,197,1.051083,0.044910,0.730933,-2.212990,-0.463833,0.715052,-0.026129,1.511854,1.231683,...,-0.969985,0.141937,0.395766,-0.028864,0.389493,0.287687,1.224717,0.148711,1.038427,1.325329
197,198,0.741301,-0.034300,-0.513853,-1.334015,-0.241736,0.714894,1.273530,1.503422,0.952709,...,0.336294,0.508108,-0.250499,-0.066795,1.209422,0.750171,0.445647,1.769830,0.516658,0.726562
198,199,0.645178,-0.863887,1.622104,-0.014481,0.438638,1.509175,1.181227,1.642535,1.141339,...,1.153573,0.771389,-0.741893,-1.417714,1.988426,0.697544,0.164035,0.910988,0.881662,1.762164


In [52]:
big_df.to_excel('../wscore_data.xlsx')